# One-Class Anomaly Detection on CIFAR-10 (ConvAE + CBAM)

This project studies **one-class / unsupervised anomaly detection** on images using a
**Convolutional Autoencoder (ConvAE)** trained only on *normal* samples from CIFAR-10.
An attention mechanism (**CBAM: Channel & Spatial Attention**) is integrated to improve
feature focusing and provide more interpretable anomaly cues.

## Key Idea
- Train an autoencoder **only on one normal class** (default: `airplane`).
- At test time:
  - **OOD detection**: treat other CIFAR-10 classes as anomalies (image-level).
  - **Synthetic localization**: inject synthetic anomalies into normal images (pixel-level masks).

## Model
- Encoder–Decoder ConvAE
- CBAM inserted at the bottleneck:
  - Channel Attention
  - Spatial Attention (also used as a soft weighting for anomaly scoring)

## Outputs
When you run evaluation, the script prints:
- `AUROC recon_mean`
- `AUROC recon_att_weighted`
- `AUROC fused(rank)`
- Localization metrics on synthetic anomalies:
  - IoU/Dice with fixed threshold (0.5)
  - IoU/Dice with quantile threshold

It also saves:
- `./runs/vis_final.png` (qualitative visualization grid)

## Requirements
- Python 3.9+
- PyTorch + torchvision
- matplotlib
- numpy


One-Class Image Anomaly Detection using ConvAE + CBAM

This script implements an unsupervised / one-class anomaly detection
framework on the CIFAR-10 dataset.

Core ideas:
- Train a convolutional autoencoder (ConvAE) using only normal samples.
- Integrate CBAM (Channel & Spatial Attention) at the bottleneck layer.
- Detect anomalies via reconstruction error.
- Enhance anomaly scoring using attention-weighted reconstruction.
- Evaluate both:
  (1) Image-level anomaly detection (AUROC on OOD samples)
  (2) Pixel-level anomaly localization using synthetic anomalies

Key Features:
- One-class training (no anomaly labels used in training)
- Synthetic anomaly generation for localization evaluation
- Attention-aware anomaly scoring
- CPU-friendly evaluation setup
- Qualitative visualization of anomaly maps and masks

Outputs:
- AUROC scores (mean, attention-weighted, fused)
- IoU / Dice scores for localization
- Visualization image saved to ./runs/vis_final.png

Dataset:
- CIFAR-10 (single normal class, default: airplane)

Author:
- Rashin Gholijani Farahani

Course:
- Deep Learning



In [1]:
# Configuration

In [19]:
import os
import math
import time
import random
from dataclasses import dataclass
from typing import Tuple, Dict, Optional

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF

import matplotlib.pyplot as plt

In [2]:
# 2) Config

@dataclass
class CFG:
    # Paths
    root: str = "./data"
    save_dir: str = "./runs"
    ckpt_name: str = "best_cpu_fast.pt"

    # One-class setup
    normal_class: int = 0
    seed: int = 42

    # Model
    base_ch: int = 24
    dropout_p: float = 0.10

    # Data loading
    batch_train: int = 32
    batch_test: int = 128
    num_workers: int = 0
    torch_threads: int = 2

    # Evaluation limits (for quick CPU evaluation)
    eval_ood_max: int = 5000
    eval_synth_max: int = 1000

    # Synthetic anomaly parameters
    synth_prob: float = 1.0
    patch_area: Tuple[float, float] = (0.03, 0.14)  # fraction of image area
    noise_std: Tuple[float, float] = (0.18, 0.45)

    # Score fusion
    fuse_w: float = 0.65  # weight for mean recon score in fused rank score

    # Localization post-processing
    smooth_k: int = 3
    quantile_q: float = 0.95


In [3]:
# 3) Small utilities


def seed_everything(seed: int) -> None:
    """Make runs more reproducible (still not perfect, but good enough for CPU eval)."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


def _rand_rect(h: int, w: int, area_range: Tuple[float, float]) -> Tuple[int, int, int, int]:
    """
    Sample a random rectangle (top, left, height, width) given an area range.

    - area_range is a fraction of total image area.
    - aspect ratio is randomized to avoid only-square patches.
    """
    area = random.uniform(area_range[0], area_range[1]) * (h * w)
    aspect = random.uniform(0.6, 1.6)

    rh = int(round(math.sqrt(area * aspect)))
    rw = int(round(math.sqrt(area / aspect)))

    # keep it valid
    rh = max(2, min(rh, h - 1))
    rw = max(2, min(rw, w - 1))

    top = random.randint(0, h - rh)
    left = random.randint(0, w - rw)
    return top, left, rh, rw


def synth_anomaly(x: torch.Tensor, cfg: CFG) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Create a synthetic anomaly on a single image tensor x (C,H,W) in [0,1].
    Returns:
      xa: anomalous image
      m : binary anomaly mask (1,H,W) where 1 indicates anomaly region
    """
    c, h, w = x.shape

    # Sometimes, do nothing (if synth_prob < 1)
    if random.random() > cfg.synth_prob:
        m = torch.zeros(1, h, w, dtype=torch.float32)
        return x.clone(), m

    top, left, rh, rw = _rand_rect(h, w, cfg.patch_area)
    m = torch.zeros(1, h, w, dtype=torch.float32)
    m[:, top:top + rh, left:left + rw] = 1.0

    # Option A: copy-paste a random patch from the same image (harder to detect)
    if random.random() < 0.5:
        src_top = random.randint(0, h - rh)
        src_left = random.randint(0, w - rw)

        patch = x[:, src_top:src_top + rh, src_left:src_left + rw].clone()
        xa = x.clone()
        xa[:, top:top + rh, left:left + rw] = patch
        return xa, m

    # Option B: add random noise in the patch
    std = random.uniform(cfg.noise_std[0], cfg.noise_std[1])
    noise = torch.randn((c, rh, rw), dtype=x.dtype) * std

    xa = x.clone()
    xa[:, top:top + rh, left:left + rw] = torch.clamp(
        xa[:, top:top + rh, left:left + rw] + noise, 0.0, 1.0
    )
    return xa, m


In [4]:

# 4) Dataset (One-class CIFAR10)

class OneClassCIFAR(Dataset):
    """
    Splits:
      - train      : only normal class images, returns x
      - test_ood   : full CIFAR10 test, returns (x, y_is_anom)
      - test_synth : only normal class test images, returns (x_anom, mask)
    """

    def __init__(self, root: str, split: str, cfg: CFG, download: bool = True):
        assert split in {"train", "test_ood", "test_synth"}
        self.split = split
        self.cfg = cfg

        train_flag = split == "train"
        base = datasets.CIFAR10(root=root, train=train_flag, download=download)

        self.data = base.data
        self.targets = np.array(base.targets, dtype=np.int64)

        # For one-class training (and synthetic eval), keep only normal class
        if split in {"train", "test_synth"}:
            idx = np.where(self.targets == cfg.normal_class)[0]
            self.data = self.data[idx]
            self.targets = self.targets[idx]

        self.to_tensor = transforms.ToTensor()
        self.train_aug = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomCrop(32, padding=2, padding_mode="reflect"),
        ])

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, i: int):
        img = TF.to_pil_image(self.data[i])

        if self.split == "train":
            x = self.to_tensor(self.train_aug(img))
            return x

        if self.split == "test_ood":
            x = self.to_tensor(img)
            y_is_anom = 0 if int(self.targets[i]) == self.cfg.normal_class else 1
            return x, torch.tensor(y_is_anom, dtype=torch.long)

        if self.split == "test_synth":
            xc = self.to_tensor(img)
            xa, m = synth_anomaly(xc, self.cfg)
            return xa, m

        raise RuntimeError("Invalid split")




In [5]:
# 5) Attention blocks (CBAM)

class ChannelAttention(nn.Module):
    """Channel attention via (avg + max) pooled descriptors."""
    def __init__(self, ch: int, r: int = 8):
        super().__init__()
        hidden = max(4, ch // r)
        self.mlp = nn.Sequential(
            nn.Conv2d(ch, hidden, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden, ch, 1, bias=False),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        avg = F.adaptive_avg_pool2d(x, 1)
        mx = F.adaptive_max_pool2d(x, 1)
        w = torch.sigmoid(self.mlp(avg) + self.mlp(mx))
        return x * w


class SpatialAttention(nn.Module):
    """Spatial attention via conv over [channel-avg, channel-max]."""
    def __init__(self, k: int = 7):
        super().__init__()
        p = (k - 1) // 2
        self.conv = nn.Conv2d(2, 1, kernel_size=k, padding=p, bias=False)

    def forward(self, x: torch.Tensor):
        avg = torch.mean(x, dim=1, keepdim=True)
        mx, _ = torch.max(x, dim=1, keepdim=True)
        a = torch.sigmoid(self.conv(torch.cat([avg, mx], dim=1)))
        return x * a, a  # return both modulated features and attention map


class CBAM(nn.Module):
    """Convolutional Block Attention Module: Channel + Spatial."""
    def __init__(self, ch: int, r: int = 8):
        super().__init__()
        self.ca = ChannelAttention(ch, r=r)
        self.sa = SpatialAttention(k=7)

    def forward(self, x: torch.Tensor):
        x = self.ca(x)
        x, a = self.sa(x)
        return x, a



In [9]:
# 6) Main model: Conv AutoEncoder + CBAM

class ConvAE_CBAM(nn.Module):
    """
    Simple Conv AutoEncoder with CBAM at bottleneck.
    Outputs:
      - reconstruction
      - attention map (low-res, later upsampled for scoring)
    """
    def __init__(self, in_ch: int = 3, base: int = 24, dropout_p: float = 0.10):
        super().__init__()

        # Encoder
        self.e1 = nn.Sequential(
            nn.Conv2d(in_ch, base, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(base, base, 3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.e2 = nn.Sequential(
            nn.Conv2d(base, base * 2, 4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(base * 2, base * 2, 3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.e3 = nn.Sequential(
            nn.Conv2d(base * 2, base * 3, 4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(base * 3, base * 3, 3, padding=1),
            nn.ReLU(inplace=True),
        )

        # Bottleneck attention + dropout (only during training)
        self.cbam = CBAM(base * 3, r=8)
        self.drop = nn.Dropout2d(p=dropout_p)

        # Decoder
        self.d3 = nn.Sequential(
            nn.ConvTranspose2d(base * 3, base * 2, 4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(base * 2, base * 2, 3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.d2 = nn.Sequential(
            nn.ConvTranspose2d(base * 2, base, 4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(base, base, 3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.out = nn.Sequential(
            nn.Conv2d(base, in_ch, 3, padding=1),
            nn.Sigmoid(),  # inputs are in [0,1]
        )

    def forward(self, x: torch.Tensor):
        z = self.e1(x)
        z = self.e2(z)
        z = self.e3(z)

        z, att = self.cbam(z)
        if self.training:
            z = self.drop(z)

        y = self.d3(z)
        y = self.d2(y)
        y = self.out(y)
        return y, att



In [10]:
# 7) Metrics helpers

def auroc(scores: np.ndarray, labels: np.ndarray) -> float:
    """
    Lightweight AUROC (no sklearn). Assumes labels are {0,1}.
    """
    order = np.argsort(scores)
    ranks = np.empty_like(order)
    ranks[order] = np.arange(len(scores))

    pos = labels == 1
    n_pos = int(pos.sum())
    n_neg = int((~pos).sum())
    if n_pos == 0 or n_neg == 0:
        return float("nan")

    sum_ranks_pos = float(ranks[pos].sum())
    return (sum_ranks_pos - n_pos * (n_pos - 1) / 2.0) / (n_pos * n_neg)


def rank01(x: torch.Tensor) -> torch.Tensor:
    """Turn a vector into rank-based [0,1] values (robust for fusing different scales)."""
    r = torch.argsort(torch.argsort(x)).float()
    return r / (r.max() + 1e-8)


def iou_dice(pred: torch.Tensor, gt: torch.Tensor, eps: float = 1e-8) -> Tuple[float, float]:
    """Compute mean IoU and Dice over a batch. pred/gt expected shape (B,1,H,W)."""
    pred = pred.float()
    gt = gt.float()

    inter = (pred * gt).sum(dim=(1, 2, 3))
    union = (pred + gt - pred * gt).sum(dim=(1, 2, 3))

    iou = (inter + eps) / (union + eps)
    dice = (2 * inter + eps) / (pred.sum(dim=(1, 2, 3)) + gt.sum(dim=(1, 2, 3)) + eps)
    return float(iou.mean().item()), float(dice.mean().item())



In [11]:
# 8) Evaluation: OOD detection

@torch.no_grad()
def eval_ood_scores(
    model: nn.Module,
    loader: DataLoader,
    device: torch.device,
    max_eval: int,
    fuse_w: float
):
    """
    Returns AUROC for:
      - mean recon error
      - attention-weighted recon error
      - fused rank score
    """
    model.eval()
    scores_mean, scores_att, labels = [], [], []
    seen = 0

    for x, y in loader:
        x = x.to(device)
        xhat, att = model(x)

        # Per-pixel recon error (L1)
        err = torch.mean(torch.abs(x - xhat), dim=1, keepdim=True)

        # Simple image-level score
        mean_score = err.mean(dim=(1, 2, 3)).cpu()

        # Attention-weighted score (upsample attention to 32x32)
        att_up = F.interpolate(att, size=(32, 32), mode="bilinear", align_corners=False)
        w_map = 1.0 + att_up
        att_score = (err * w_map).sum(dim=(1, 2, 3)) / (w_map.sum(dim=(1, 2, 3)) + 1e-8)
        att_score = att_score.cpu()

        scores_mean.append(mean_score)
        scores_att.append(att_score)
        labels.append(y.cpu())

        seen += x.size(0)
        if seen >= max_eval:
            break

    s_mean = torch.cat(scores_mean, dim=0)[:max_eval]
    s_att = torch.cat(scores_att, dim=0)[:max_eval]
    y = torch.cat(labels, dim=0)[:max_eval].numpy()

    auc_mean = auroc(s_mean.numpy(), y)
    auc_att = auroc(s_att.numpy(), y)

    fused = fuse_w * rank01(s_mean) + (1.0 - fuse_w) * rank01(s_att)
    auc_fused = auroc(fused.numpy(), y)

    return auc_mean, auc_att, auc_fused



In [12]:
# 9) Evaluation: Synthetic localization

@torch.no_grad()
def eval_synth_localization(
    model: nn.Module,
    loader: DataLoader,
    device: torch.device,
    smooth_k: int,
    q: float
):
    """
    For synthetic anomalies, compute localization quality:
      - threshold 0.5 on normalized anomaly map
      - threshold using per-image quantile q
    """
    model.eval()
    iou05, dice05 = [], []
    iouq, diceq = [], []

    for xa, m in loader:
        xa = xa.to(device)
        m = m.to(device)

        xhat, _ = model(xa)

        # Build anomaly map from max channel abs error
        a_map = torch.max(torch.abs(xa - xhat), dim=1, keepdim=True)[0]

        # Smooth (helps reduce salt-and-pepper noise)
        a_map = F.avg_pool2d(a_map, kernel_size=smooth_k, stride=1, padding=smooth_k // 2)

        # Normalize to [0,1] per image
        a_min = a_map.amin(dim=(1, 2, 3), keepdim=True)
        a_max = a_map.amax(dim=(1, 2, 3), keepdim=True)
        a_map01 = (a_map - a_min) / (a_max - a_min + 1e-8)

        # Fixed threshold
        pred05 = (a_map01 > 0.5).float()
        i, d = iou_dice(pred05, m)
        iou05.append(i)
        dice05.append(d)

        # Quantile threshold (adapts to each image)
        th = torch.quantile(a_map01.flatten(1), q, dim=1).view(-1, 1, 1, 1)
        predq = (a_map01 > th).float()
        i, d = iou_dice(predq, m)
        iouq.append(i)
        diceq.append(d)

    return (float(np.mean(iou05)), float(np.mean(dice05))), (float(np.mean(iouq)), float(np.mean(diceq)))


In [13]:
# 10) DataLoaders

def make_loaders(cfg: CFG) -> Dict[str, DataLoader]:
    train_ds = OneClassCIFAR(cfg.root, "train", cfg, download=True)
    ood_ds = OneClassCIFAR(cfg.root, "test_ood", cfg, download=True)
    synth_ds = OneClassCIFAR(cfg.root, "test_synth", cfg, download=True)

    # Optional subsampling for faster eval on CPU
    if cfg.eval_ood_max and cfg.eval_ood_max < len(ood_ds):
        idx = np.random.RandomState(cfg.seed).choice(len(ood_ds), cfg.eval_ood_max, replace=False)
        ood_ds = Subset(ood_ds, idx.tolist())

    if cfg.eval_synth_max and cfg.eval_synth_max < len(synth_ds):
        idx = np.random.RandomState(cfg.seed + 1).choice(len(synth_ds), cfg.eval_synth_max, replace=False)
        synth_ds = Subset(synth_ds, idx.tolist())

    train_loader = DataLoader(
        train_ds,
        batch_size=cfg.batch_train,
        shuffle=True,
        drop_last=True,
        num_workers=cfg.num_workers,
    )
    ood_loader = DataLoader(
        ood_ds,
        batch_size=cfg.batch_test,
        shuffle=False,
        num_workers=cfg.num_workers,
    )
    synth_loader = DataLoader(
        synth_ds,
        batch_size=cfg.batch_test,
        shuffle=False,
        num_workers=cfg.num_workers,
    )

    return {"train": train_loader, "ood": ood_loader, "synth": synth_loader}




In [14]:
# 10) DataLoaders
def make_loaders(cfg: CFG) -> Dict[str, DataLoader]:
    train_ds = OneClassCIFAR(cfg.root, "train", cfg, download=True)
    ood_ds = OneClassCIFAR(cfg.root, "test_ood", cfg, download=True)
    synth_ds = OneClassCIFAR(cfg.root, "test_synth", cfg, download=True)

    # Optional subsampling for faster eval on CPU
    if cfg.eval_ood_max and cfg.eval_ood_max < len(ood_ds):
        idx = np.random.RandomState(cfg.seed).choice(len(ood_ds), cfg.eval_ood_max, replace=False)
        ood_ds = Subset(ood_ds, idx.tolist())

    if cfg.eval_synth_max and cfg.eval_synth_max < len(synth_ds):
        idx = np.random.RandomState(cfg.seed + 1).choice(len(synth_ds), cfg.eval_synth_max, replace=False)
        synth_ds = Subset(synth_ds, idx.tolist())

    train_loader = DataLoader(
        train_ds,
        batch_size=cfg.batch_train,
        shuffle=True,
        drop_last=True,
        num_workers=cfg.num_workers,
    )
    ood_loader = DataLoader(
        ood_ds,
        batch_size=cfg.batch_test,
        shuffle=False,
        num_workers=cfg.num_workers,
    )
    synth_loader = DataLoader(
        synth_ds,
        batch_size=cfg.batch_test,
        shuffle=False,
        num_workers=cfg.num_workers,
    )

    return {"train": train_loader, "ood": ood_loader, "synth": synth_loader}



In [20]:
@torch.no_grad()
def save_visual_examples(model, synth_loader, device, out_path, n=8, smooth_k=3):
    model.eval()
    rows = n
    cols = 5

    xs, xhats, amaps, preds, gts = [], [], [], [], []

    for xa, m in synth_loader:
        xa = xa.to(device)
        m = m.to(device)

        xhat, _ = model(xa)

        a_map = torch.max(torch.abs(xa - xhat), dim=1, keepdim=True)[0]
        a_map = F.avg_pool2d(a_map, kernel_size=smooth_k, stride=1, padding=smooth_k // 2)

        a_map01 = (a_map - a_map.amin(dim=(1, 2, 3), keepdim=True)) / (
            a_map.amax(dim=(1, 2, 3), keepdim=True) - a_map.amin(dim=(1, 2, 3), keepdim=True) + 1e-8
        )

        pred = (a_map01 > 0.5).float()

        for i in range(xa.size(0)):
            xs.append(xa[i].detach().cpu())
            xhats.append(xhat[i].detach().cpu())
            amaps.append(a_map01[i].detach().cpu())
            preds.append(pred[i].detach().cpu())
            gts.append(m[i].detach().cpu())
            if len(xs) >= n:
                break
        if len(xs) >= n:
            break

    def to_img(x3):
        x = x3.permute(1, 2, 0).numpy()
        return np.clip(x, 0, 1)

    fig = plt.figure(figsize=(cols * 3.2, rows * 3.0))
    for r in range(rows):
        xa = xs[r]
        xhat = xhats[r]
        amap = amaps[r][0].numpy()
        pm = preds[r][0].numpy()
        gm = gts[r][0].numpy()

        items = [
            ("Input (synthetic)", to_img(xa)),
            ("Reconstruction", to_img(xhat)),
            ("Anomaly map", amap),
            ("Pred mask", pm),
            ("GT mask", gm),
        ]

        for c in range(cols):
            ax = fig.add_subplot(rows, cols, r * cols + c + 1)
            title, im = items[c]
            if c <= 1:
                ax.imshow(im)
            else:
                ax.imshow(im, cmap="gray", vmin=0, vmax=1)
            ax.set_title(title, fontsize=10)
            ax.axis("off")

    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    plt.tight_layout()
    plt.savefig(out_path, dpi=200)
    plt.close()
    print("saved visualization:", out_path)


def main():
    cfg = CFG()
    seed_everything(cfg.seed)
    torch.set_num_threads(cfg.torch_threads)

    device = torch.device("cpu")
    loaders = make_loaders(cfg)

    model = ConvAE_CBAM(in_ch=3, base=cfg.base_ch, dropout_p=cfg.dropout_p).to(device)

    ckpt_path = os.path.join(cfg.save_dir, cfg.ckpt_name)
    if not os.path.exists(ckpt_path):
        raise FileNotFoundError(f"Checkpoint not found: {ckpt_path}")

    model.load_state_dict(torch.load(ckpt_path, map_location=device))
    print("loaded:", ckpt_path)

    t0 = time.time()

    auc_mean, auc_att, auc_fused = eval_ood_scores(
        model, loaders["ood"], device, max_eval=cfg.eval_ood_max, fuse_w=cfg.fuse_w
    )

    (iou05, dice05), (iouq, diceq) = eval_synth_localization(
        model, loaders["synth"], device, smooth_k=cfg.smooth_k, q=cfg.quantile_q
    )

    print(f"AUROC recon_mean        : {auc_mean:.4f}")
    print(f"AUROC recon_att_weighted: {auc_att:.4f}")
    print(f"AUROC fused(rank)       : {auc_fused:.4f}")
    print(f"Localization th=0.5     : IoU {iou05:.4f} | Dice {dice05:.4f}")
    print(f"Localization quantile   : IoU {iouq:.4f} | Dice {diceq:.4f}")
    print("time:", round(time.time() - t0, 1), "s")

    save_visual_examples(
        model,
        loaders["synth"],
        device,
        out_path=os.path.join(cfg.save_dir, "vis_final.png"),
        n=8,
        smooth_k=cfg.smooth_k
    )

# This block ensures that the evaluation pipeline is executed
# only when this file is run directly, and not when imported
# as a module in another script.

if __name__ == "__main__":
    main()



Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
loaded: ./runs/best_cpu_fast.pt
AUROC recon_mean        : 0.7071
AUROC recon_att_weighted: 0.7070
AUROC fused(rank)       : 0.7071
Localization th=0.5     : IoU 0.4764 | Dice 0.5604
Localization quantile   : IoU 0.3476 | Dice 0.4560
time: 9.3 s
saved visualization: ./runs/vis_final.png


## Author:

# Rashin Gholijani Farahani - 2025-2026